In [2]:
!pip install gym[atari,toy_text,accept_rom_license]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 79.6 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.6.0
    Uninstalling pygame-2.6.0:
      Successfully uninstalled pygame-2.6.0


In [1]:
import numpy as np
import random
import gym
env = gym.make('Taxi-v3', new_step_api=True)

# step types
STEPTYPE_FIRST = 0
STEPTYPE_MID = 1
STEPTYPE_LAST = 2

Q = np.random.uniform(size=(500, 6))

In [2]:
env.reset()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


91

In [3]:
env.step(0)

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


(191,
 -1,
 False,
 False,
 {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})

In [4]:
# wrapper for gym's blackjack environment
def generate_start_step():
    return { 'observation': env.reset(), 'reward': 0., 'step_type': STEPTYPE_FIRST }

def generate_next_step(step, action):
    obs, reward, done, _, info = env.step(action)
    step_type = STEPTYPE_LAST if done else STEPTYPE_MID
    return { 'observation': obs, 'reward': reward, 'step_type': step_type }

In [5]:
epsilon = 0.1

def get_eps_soft_action(step):
    # epsilon-soft greedy policy
    if random.uniform(0, 1) < epsilon:
        # epsilon 확률로 무작위 행동 선택
        return env.action_space.sample()
    else:
        # (1 - epsilon) 확률로 Q 값이 가장 큰 행동 선택
        observation = step['observation']
        return np.argmax(Q[observation])


In [6]:
def get_greedy_action(step):
    observ = step['observation']
    return np.argmax(Q[observ])

In [7]:
def get_random_action(step):
    return random.randint(0, env.action_space.n-1)

behavior_prob_hit = 1. / float(env.action_space.n)

In [8]:
# return true if (observ, action) exists in epi
def in_episode(epi, observ, action):
    for s, a in zip(*epi):
        if s['observation'] == observ and a == action:
            return True
    return False

In [9]:
def generate_episode(policy_func=get_random_action):
    episode = list()
    actions = list()
    frames = list()
    step = generate_start_step()
    frames.append(env.render(mode='ansi'))
    episode.append(step)
    while step['step_type'] != STEPTYPE_LAST:
        action = policy_func(step)
        step = generate_next_step(step, action)
        frames.append(env.render(mode='ansi'))
        episode.append(step)
        actions.append(action)
    return episode, actions, frames

In [10]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame)
        sleep(.2)

In [11]:
maxiter = 100000
gamma = 1
epsilon = 0.3
lr_rate = 0.8

Q = np.random.uniform(size=(env.observation_space.n, env.action_space.n))

for _ in range(maxiter):
    # starting step
    step = generate_start_step()
    action = get_random_action(step)
    done = False
    while not done:
        next_step = generate_next_step(step, action)

        if next_step['step_type'] == STEPTYPE_LAST:
            state = step['observation']
            idx1 = (state, action)
            Q[idx1] = Q[idx1] + lr_rate * (next_step['reward'] - Q[idx1])
            done = True
        else:
            best_action = get_greedy_action(next_step)

            state = step['observation']
            next_state = next_step['observation']
            idx1 = (state, action)
            idx2 = (next_state, best_action)
            Q[idx1] = Q[idx1] + lr_rate * ((next_step['reward'] + gamma * Q[idx2]) - Q[idx1])

            next_action = get_eps_soft_action(step)

            step = next_step
            action = next_action


In [17]:
epi, actions, frames = generate_episode(policy_func=get_greedy_action)
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



In [3]:
import numpy as np
import random
import gym
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

env = CliffWalkingEnv()

# step types
STEPTYPE_FIRST = 0
STEPTYPE_MID = 1
STEPTYPE_LAST = 2

Q = np.random.uniform(size=(env.observation_space.n, env.action_space.n))

def generate_start_step():
    return {'observation': env.reset(), 'reward': 0., 'step_type': STEPTYPE_FIRST}

def generate_next_step(step, action):
    obs, reward, done, _, info = env.step(action)
    step_type = STEPTYPE_LAST if done else STEPTYPE_MID
    return {'observation': obs, 'reward': reward, 'step_type': step_type}

epsilon = 0.1

def get_eps_soft_action(step):
    if random.uniform(0, 1) < epsilon:
        return env.action_space.sample()
    else:
        observation = step['observation']
        return np.argmax(Q[observation])

def get_greedy_action(state):
    return np.argmax(Q[state])

def generate_episode(policy_func):
    step = generate_start_step()
    frames = []
    done = False

    while not done:
        state = step['observation']
        action = policy_func(state)
        next_step = generate_next_step(step, action)

        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': next_step['reward']
        })

        step = next_step
        done = next_step['step_type'] == STEPTYPE_LAST

    return frames

def print_frames(frames):
    for i, frame_info in enumerate(frames):
        clear_output(wait=True)
        print(frame_info['frame'])
        print(f"Timestep: {i+1}")
        print(f"State: {frame_info['state']}")
        print(f"Action: {frame_info['action']}")
        print(f"Reward: {frame_info['reward']}")
        time.sleep(1)

# Hyperparameters
lr_rate = 0.1
gamma = 0.99

# Training Q-learning
num_episodes = 500
reward_per_episode = []

for episode in range(num_episodes):
    step = generate_start_step()
    done = False
    total_reward = 0

    while not done:
        action = get_eps_soft_action(step)
        next_step = generate_next_step(step, action)

        state = step['observation']
        next_state = next_step['observation']
        reward = next_step['reward']
        done = next_step['step_type'] == STEPTYPE_LAST

        best_action = np.argmax(Q[next_state])
        Q[state, action] = Q[state, action] + lr_rate * (reward + gamma * Q[next_state, best_action] - Q[state, action])

        step = next_step
        total_reward += reward

    reward_per_episode.append(total_reward)

print("Training completed")

# Plotting the training progress
plt.plot(reward_per_episode)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('Total Reward per Episode during Training')
plt.show()

# Generate and print frames for a sample episode


o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  x


Timestep: 13
State: 35
Action: 2
Reward: -1


In [4]:
frames = generate_episode(policy_func=get_greedy_action)
print_frames(frames)


o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  x


Timestep: 13
State: 35
Action: 2
Reward: -1
